In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/competitions/playground-series-s5e8/sample_submission.csv
/kaggle/input/competitions/playground-series-s5e8/train.csv
/kaggle/input/competitions/playground-series-s5e8/test.csv


In [2]:
from sklearn.ensemble import RandomForestClassifier

print("Libraries imported!")

Libraries imported!


In [3]:
train = pd.read_csv("/kaggle/input/competitions/playground-series-s5e8/train.csv")
test = pd.read_csv("/kaggle/input/competitions/playground-series-s5e8/test.csv")

print("Train shape:", train.shape)
print("Test shape:", test.shape)

Train shape: (750000, 18)
Test shape: (250000, 17)


In [4]:
train.head()

,id,age,job,marital,education,default,balance,housing,loan,contact,day,month,duration,campaign,pdays,previous,poutcome,y
0,0,42,technician,married,secondary,no,7,no,no,cellular,25,aug,117,3,-1,0,unknown,0
1,1,38,blue-collar,married,secondary,no,514,no,no,unknown,18,jun,185,1,-1,0,unknown,0
2,2,36,blue-collar,married,secondary,no,602,yes,no,unknown,14,may,111,2,-1,0,unknown,0
3,3,27,student,single,secondary,no,34,yes,no,unknown,28,may,10,2,-1,0,unknown,0
4,4,26,technician,married,secondary,no,889,yes,no,cellular,3,feb,902,1,-1,0,unknown,1


In [5]:
train.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 750000 entries, 0 to 749999
Data columns (total 18 columns):
 #   Column     Non-Null Count   Dtype 
---  ------     --------------   ----- 
 0   id         750000 non-null  int64 
 1   age        750000 non-null  int64 
 2   job        750000 non-null  object
 3   marital    750000 non-null  object
 4   education  750000 non-null  object
 5   default    750000 non-null  object
 6   balance    750000 non-null  int64 
 7   housing    750000 non-null  object
 8   loan       750000 non-null  object
 9   contact    750000 non-null  object
 10  day        750000 non-null  int64 
 11  month      750000 non-null  object
 12  duration   750000 non-null  int64 
 13  campaign   750000 non-null  int64 
 14  pdays      750000 non-null  int64 
 15  previous   750000 non-null  int64 
 16  poutcome   750000 non-null  object
 17  y          750000 non-null  int64 
dtypes: int64(9), object(9)
memory usage: 103.0+ MB


In [6]:
train.dtypes.value_counts()


int64     9
object    9
Name: count, dtype: int64

In [7]:
train.isnull().sum()

id           0
age          0
job          0
marital      0
education    0
default      0
balance      0
housing      0
loan         0
contact      0
day          0
month        0
duration     0
campaign     0
pdays        0
previous     0
poutcome     0
y            0
dtype: int64

In [8]:
y = train["y"]


train_features = train.drop(columns=["id", "y"])


test_features = test.drop(columns=["id"])



In [9]:
combined = pd.concat([train_features, test_features], axis=0)

print("Combined shape:", combined.shape)

Combined shape: (1000000, 16)


In [10]:
combined_encoded = pd.get_dummies(combined)


print("Encoded shape:", combined_encoded.shape)

Encoded shape: (1000000, 51)


In [11]:
X = combined_encoded.iloc[:len(train)]
X_test = combined_encoded.iloc[len(train):]

print("Final train shape:", X.shape)
print("Final test shape:", X_test.shape)

Final train shape: (750000, 51)
Final test shape: (250000, 51)


In [12]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score

X_train, X_val, y_train, y_val = train_test_split(
    X, y,
    test_size=0.2,
    random_state=42,
    stratify=y
)

print("Train split:", X_train.shape)
print("Validation split:", X_val.shape)

Train split: (600000, 51)
Validation split: (150000, 51)


In [13]:
model = RandomForestClassifier(
    n_estimators=100,
    random_state=42,
    n_jobs=-1
)

model.fit(X_train, y_train)

print("Model trained ")

Model trained 


In [14]:
val_preds = model.predict_proba(X_val)[:, 1]
val_score = roc_auc_score(y_val, val_preds)

print("Validation AUC:", val_score)

Validation AUC: 0.9607460497212021


In [15]:
model.fit(X, y)

RandomForestClassifier(n_jobs=-1, random_state=42)

In [16]:
predictions = model.predict_proba(X_test)[:, 1]

print("Predictions ")

Predictions 


In [17]:
submission = pd.DataFrame({
    "id": test["id"],
    "y": predictions
})

In [18]:
submission.to_csv("submission.csv", index=False)


print("Submission file ")
print("Location: /kaggle/working/submission.csv")


Submission file 
Location: /kaggle/working/submission.csv
